<a href="https://colab.research.google.com/github/alexcpn/tranformer_learn/blob/gpt-loss-learn/LLM_Loss_Understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.5 MB/s eta 0:00:00


# Our small Training and Target data

In [2]:
input_text  = "Welcome to New York Zoo" # As New York City is most common and the pre-trained model may predict it
target_text = input_text

# Load Model, Tokenizer etc

In [3]:
# from Karpathy and modified
# https://github.com/karpathy/nanoGPT/blob/086ebe1822791b775e951b4b562fbb7131d83cc2/train.py
def get_batch(len_train_data,input_ids,attention_mask,device,block_size,
                    batch_size):
    #print(f"len_train_data={len_train_data} block_size ={block_size} batch_size= {batch_size}")

    if len_train_data > block_size:
      ix = torch.randint(0,len_train_data-block_size , (batch_size,)) # random select from training data set
    else:
     ix = torch.zeros(batch_size, dtype=torch.int) # else give all data as is but in batches
    #print(f"ix {ix.shape} ={ix.tolist()}")
    x = torch.stack([(input_ids[i:i+block_size]) for i in ix])
    y = torch.stack([((attention_mask[i:i+block_size])) for i in ix])
    #print(x.shape,x)
    # # here is the encoding
    # torch.Size([2, 5]) tensor([[14618,   284,   968,  1971, 21980],
    #    [14618,   284,   968,  1971, 21980]])
    x, y = x.to(device), y.to(device)
    return x, y


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import shutil
from transformers import  get_linear_schedule_with_warmup # for training
from datetime import datetime
import re
import torch._dynamo.config


model_name = 'gpt2'

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
#print(TokenizerDetails(tokenizer) # model_max_length: 1024 # vocab_size: 50257
model = GPT2LMHeadModel.from_pretrained(model_name)


In [5]:
# USE CPU if GPU is not available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

# Eval Model before Training

In [6]:
model.eval()

# encode the inputs
encoding = tokenizer(input_text,padding=True,truncation=True,return_tensors="pt",)
input_ids, attention_mask = encoding.input_ids, encoding.attention_mask
# encode the targets
target_encoding = tokenizer(target_text,padding=True,truncation=True,return_tensors="pt",)
labels = target_encoding.input_ids
# replace padding token id's of the labels by -100 so it's ignored by the loss
labels[labels == tokenizer.pad_token_id] = -100  # in our case there is no padding
print(f"input_ids={input_ids}")
print(f"attention_mask={attention_mask}") # all ones
print(f"labels ={labels}")
# forward pass
outputs = model(input_ids=input_ids.to(device),labels=labels.to(device))
print(f"Model Loss Before training for the Target {outputs.loss}")
# Test the model to check what it predicts next
# remove the last token off for input-id's as well as attention Mask
input_ids = input_ids[:,:-1] # input_text  = "Welcome to New York"
attention_mask = attention_mask[:,:-1]
print(f"input_ids={input_ids}")
outputs = model.generate(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device),max_new_tokens=1)
answer = tokenizer.decode(outputs[0], skip_special_tokens=False)
print(f"Result '{answer}'")

input_ids=tensor([[14618,   284,   968,  1971, 21980]])
attention_mask=tensor([[1, 1, 1, 1, 1]])
labels =tensor([[14618,   284,   968,  1971, 21980]])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Model Loss Before training for the Target 4.441195011138916
input_ids=tensor([[14618,   284,   968,  1971]])
Result 'Welcome to New York City'


In [32]:
import numpy as np

outputs = model.generate(input_ids=input_ids.to(device),max_new_tokens=12,output_scores=True, return_dict_in_generate=True)
transition_scores = model.compute_transition_scores(outputs.sequences, outputs.scores, normalize_logits=True)
#answer = tokenizer.decode(outputs[0], skip_special_tokens=False)

input_length = 1 if model.config.is_encoder_decoder else input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
for tok, score in zip(generated_tokens[0], transition_scores[0]):
    score = score.cpu()
    print(f"| token | token string | logits | probability")
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.numpy():.3f} | {np.exp(score.numpy()):.2%}")

#print(f"Possible ouputs '{outputs.sequences}' Scores ='{outputs.scores}' ")
print(f"Possible ouputs '{tokenizer.batch_decode(outputs.sequences,skip_special_tokens=False)}' Scores ='{outputs.scores[0]}' ")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


| token | token string | logits | probability
|  2254 |  City    | -1.261 | 28.35%
| token | token string | logits | probability
|    11 | ,        | -1.774 | 16.97%
| token | token string | logits | probability
|   810 |  where   | -1.732 | 17.70%
| token | token string | logits | probability
|   262 |  the     | -2.020 | 13.27%
| token | token string | logits | probability
|  1748 |  city    | -2.742 | 6.45%
| token | token string | logits | probability
|   318 |  is      | -1.463 | 23.15%
| token | token string | logits | probability
|  1900 |  known   | -2.483 | 8.35%
| token | token string | logits | probability
|   329 |  for     | -0.524 | 59.21%
| token | token string | logits | probability
|   663 |  its     | -0.725 | 48.44%
| token | token string | logits | probability
|  1049 |  great   | -3.667 | 2.56%
| token | token string | logits | probability
| 10808 |  restaurants | -2.899 | 5.51%
| token | token string | logits | probability
|    11 | ,        | -0.868 | 41.99%
Poss

# Train

In [33]:
model.train() # set model for training


print(f"length of dataset in words: {len(input_text):,}") #252,023

encoding = tokenizer(input_text, truncation=True, padding=True,return_tensors='pt')
print(f"encoding.input_ids.shape {encoding.input_ids.shape}")
#encoding.input_ids.shape torch.Size([1, 6])

print(f"encoding.attention_mask.shape {encoding.attention_mask.shape}")
len_train_data = encoding.input_ids.shape[1]
print(f"len_train_data = {len_train_data}")
# len_train_data = 6
 # flatten the tensor from  torch.Size([1, 6]) to  torch.Size([48735])
input_ids=encoding.input_ids.view(-1)
attention_mask=encoding.attention_mask.view(-1)
# Note , if we give truncation as False then the token sequence length goes more than model_max_length
# Token indices sequence length is longer than the specified maximum sequence length for this
#  model (23552 > 1024). Running this sequence through the model will result in indexing errors
# However we are not running through the model; We will add it to an array and train with block_size

# Load the  model

# # Freeze bottom 10 layers
# for parameter in model.parameters():
#     parameter.requires_grad = False

for i, m in enumerate(model.transformer.h):
    #Only un-freeze the last n transformer blocks
    if i >= 10:
        for parameter in m.parameters():
            parameter.requires_grad = True

for parameter in model.transformer.ln_f.parameters():
    parameter.requires_grad = True

for parameter in model.lm_head.parameters():
    parameter.requires_grad = True


model.to(device)
# learning_rate = 6e-4 # ??
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

# Set up the training parameters
train_batch_size = 1
print(f"len_train_data = {len_train_data}")
block_size = len_train_data +1
num_train_epochs = 50

# Set the optimizer and learning rate scheduler
# num_warmup_steps = 100
# max_grad_norm = 1.0
#optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
num_train_steps = len_train_data // train_batch_size * num_train_epochs
#lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_train_steps)

model.train()
for epoch in range(num_train_epochs):
    #print(f"Epoch {epoch+1} of {num_train_epochs}")
    epoch_loss = 0
    for i in range(0,len_train_data, block_size):
        # do the batch size manipulation here
        x,y= get_batch(len_train_data,input_ids,attention_mask,device,
            block_size=block_size,batch_size=train_batch_size)
        # attention_mask given by tokenize is array of ones= [1,1,..], that is attend to all tokens
        # if we do not give the parameter, the model will attend to all tokens by default
        outputs = model(input_ids=x,attention_mask=y,labels=x)
        loss = outputs.loss
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        #lr_scheduler.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch} complete. Loss: {loss.item()} ")

print(f"Epoch {epoch} complete. Loss: {loss.item()} ")


length of dataset in words: 23
encoding.input_ids.shape torch.Size([1, 5])
encoding.attention_mask.shape torch.Size([1, 5])
len_train_data = 5
len_train_data = 5
Epoch 0 complete. Loss: 5.25325870513916 
Epoch 1 complete. Loss: 3.9808201789855957 
Epoch 2 complete. Loss: 2.83502197265625 
Epoch 3 complete. Loss: 2.7266743183135986 
Epoch 4 complete. Loss: 1.1906808614730835 
Epoch 5 complete. Loss: 1.1210726499557495 
Epoch 6 complete. Loss: 0.8141047358512878 
Epoch 7 complete. Loss: 1.8467419147491455 
Epoch 8 complete. Loss: 0.25482603907585144 
Epoch 9 complete. Loss: 0.23888836801052094 
Epoch 10 complete. Loss: 0.048487767577171326 
Epoch 11 complete. Loss: 1.0369170904159546 
Epoch 12 complete. Loss: 1.0737860202789307 
Epoch 13 complete. Loss: 1.0095150470733643 
Epoch 14 complete. Loss: 0.00045086833415552974 
Epoch 15 complete. Loss: 1.0051307678222656 
Epoch 16 complete. Loss: 0.00025017300504259765 
Epoch 17 complete. Loss: 0.0031898259185254574 
Epoch 18 complete. Loss: 0.

# Eval after Training

In [34]:
model.eval()

# encode the inputs
encoding = tokenizer(input_text,padding=True,truncation=True,return_tensors="pt",)
input_ids, attention_mask = encoding.input_ids, encoding.attention_mask
# encode the targets
target_encoding = tokenizer(target_text,padding=True,truncation=True,return_tensors="pt",)
labels = target_encoding.input_ids
# replace padding token id's of the labels by -100 so it's ignored by the loss
labels[labels == tokenizer.pad_token_id] = -100  # in our case there is no padding
print(f"input_ids={input_ids}")
print(f"attention_mask={attention_mask}") # all ones
print(f"labels ={labels}")
# forward pass
outputs = model(input_ids=input_ids.to(device),labels=labels.to(device))
print(f"Model Loss After  training for the Target {outputs.loss}")
# Test the model to check what it predicts next
# remove the last token off for input-id's as well as attention Mask
input_ids = input_ids[:,:-1]
attention_mask = attention_mask[:,:-1]
print(f"input_ids={input_ids}")
outputs = model.generate(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device),max_new_tokens=1)
answer = tokenizer.decode(outputs[0], skip_special_tokens=False)
print(f"Result '{answer}'")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


input_ids=tensor([[14618,   284,   968,  1971, 21980]])
attention_mask=tensor([[1, 1, 1, 1, 1]])
labels =tensor([[14618,   284,   968,  1971, 21980]])
Model Loss After  training for the Target 0.003350791521370411
input_ids=tensor([[14618,   284,   968,  1971]])
Result 'Welcome to New York Zoo'


In [35]:
import numpy as np

outputs = model.generate(input_ids=input_ids.to(device),max_new_tokens=12,output_scores=True, return_dict_in_generate=True)
transition_scores = model.compute_transition_scores(outputs.sequences, outputs.scores, normalize_logits=True)
#answer = tokenizer.decode(outputs[0], skip_special_tokens=False)

input_length = 1 if model.config.is_encoder_decoder else input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
for tok, score in zip(generated_tokens[0], transition_scores[0]):
    score = score.cpu()
    print(f"| token | token string | logits | probability")
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.numpy():.3f} | {np.exp(score.numpy()):.2%}")

#print(f"Possible ouputs '{outputs.sequences}' Scores ='{outputs.scores}' ")
print(f"Possible ouputs '{tokenizer.batch_decode(outputs.sequences,skip_special_tokens=False)}' Scores ='{outputs.scores[0]}' ")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


| token | token string | logits | probability
| 21980 |  Zoo     | 0.000 | 100.00%
| token | token string | logits | probability
| 21980 |  Zoo     | -0.666 | 51.39%
| token | token string | logits | probability
|   290 |  and     | -0.918 | 39.92%
| token | token string | logits | probability
| 11446 |  Aqu     | -0.359 | 69.85%
| token | token string | logits | probability
| 17756 | arium    | -0.000 | 99.98%
| token | token string | logits | probability
| 21980 |  Zoo     | -0.850 | 42.76%
| token | token string | logits | probability
|   968 |  New     | -1.420 | 24.18%
| token | token string | logits | probability
|  1971 |  York    | -0.000 | 99.98%
| token | token string | logits | probability
| 21980 |  Zoo     | -0.000 | 99.97%
| token | token string | logits | probability
| 21980 |  Zoo     | -0.207 | 81.28%
| token | token string | logits | probability
| 21980 |  Zoo     | -0.657 | 51.84%
| token | token string | logits | probability
| 21980 |  Zoo     | -0.766 | 46.48%
Poss